In [2]:
import pandas as pd
import numpy
import itertools
import csv
import glob

In [3]:
# Make a list of the file names I want to iterate through using glob
epifilenames = glob.glob('epi2brad/*outfiles/*stats.txt')
print epifilenames

['epi2brad/clust_91_outfiles/clust_91_stats.txt', 'epi2brad/clust_90_outfiles/clust_90_stats.txt', 'epi2brad/clust_86_outfiles/clust_86_stats.txt', 'epi2brad/clust_87_outfiles/clust_87_stats.txt', 'epi2brad/clust_81_outfiles/clust_81_stats.txt', 'epi2brad/clust_80_outfiles/clust_80_stats.txt', 'epi2brad/clust_84_outfiles/clust_84_stats.txt', 'epi2brad/clust_92_outfiles/clust_92_stats.txt', 'epi2brad/clust_93_outfiles/clust_93_stats.txt', 'epi2brad/epi2brad_outfiles/epi2brad_stats.txt', 'epi2brad/clust_94_outfiles/clust_94_stats.txt', 'epi2brad/clust_88_outfiles/clust_88_stats.txt', 'epi2brad/clust_89_outfiles/clust_89_stats.txt', 'epi2brad/clust_82_outfiles/clust_82_stats.txt', 'epi2brad/clust_83_outfiles/clust_83_stats.txt']


In [4]:
# The *stats.txt files produced by iPyrad have two blocks of data we want to retrieve: the distribution of SNPs
# (vars and pis) per locus, and the final sample stats summary.
# Start with a list that will contain the first set of data, distribution of SNPs
snp_dist = []

# To just see line numbers within the file:
#with open("epi2brad/clust_84_outfiles/clust_84_stats.txt", "rt") as infile:
#   for line in infile:
#        lines.append(line)
#print(list(enumerate(lines)))

#for name in epifilenames:
#    open(name,'rt') # iterate through filenames generated from glob

# for now, just leave hard-coded for epifilenames[0]; happens to be clust_threshold=91

infile=open(epifilenames[0], 'rt')
counter=0
for line in infile:
    counter+=1
    if 'var  sum_var' in line:
        varline=counter
        print varline # not necessary but want to just check
        print line
    elif '## Final' in line:
        endline=counter
        print endline
        print line
    else:
        continue

infile.close() # because it's iterated through the whole file already so we need to close and reopen

#infile=open('epi2brad/clust_84_outfiles/clust_84_stats.txt','rt')
infile=open(epifilenames[0], 'rt')

#assert itertools.islice(infile, endline-2, endline-1) # throw an error if endline-2 isn't a blank line

for lines in itertools.islice(infile, varline, endline-3):
    lines2 = lines.strip().split()
    lines2.append('91') # eventually change to whatever file it's on
    snp_dist.append(lines2)
        
print snp_dist

infile.close()

snpdist_labels=['number','variable','sum_var','pis', 'sum_pis', 'clust_threshold']
df_snpdist = pd.DataFrame.from_records(snp_dist, columns=snpdist_labels)

print df_snpdist

df_snpdist.to_csv("./epi2brad_clust_91_snpdist.csv")

58
      var  sum_var     pis  sum_pis

75
## Final Sample stats summary

[['0', '64586', '0', '114007', '0', '91'], ['1', '41325', '41325', '30992', '30992', '91'], ['2', '31134', '103593', '15414', '61820', '91'], ['3', '18253', '158352', '5758', '79094', '91'], ['4', '7153', '186964', '1002', '83102', '91'], ['5', '3047', '202199', '223', '84217', '91'], ['6', '1238', '209627', '51', '84523', '91'], ['7', '477', '212966', '14', '84621', '91'], ['8', '154', '214198', '4', '84653', '91'], ['9', '66', '214792', '1', '84662', '91'], ['10', '21', '215002', '0', '84662', '91'], ['11', '6', '215068', '0', '84662', '91'], ['12', '2', '215092', '0', '84662', '91'], ['13', '4', '215144', '0', '84662', '91']]
   number variable sum_var     pis sum_pis clust_threshold
0       0    64586       0  114007       0              91
1       1    41325   41325   30992   30992              91
2       2    31134  103593   15414   61820              91
3       3    18253  158352    5758   79094           

In [5]:
print epifilenames

['epi2brad/clust_91_outfiles/clust_91_stats.txt', 'epi2brad/clust_90_outfiles/clust_90_stats.txt', 'epi2brad/clust_86_outfiles/clust_86_stats.txt', 'epi2brad/clust_87_outfiles/clust_87_stats.txt', 'epi2brad/clust_81_outfiles/clust_81_stats.txt', 'epi2brad/clust_80_outfiles/clust_80_stats.txt', 'epi2brad/clust_84_outfiles/clust_84_stats.txt', 'epi2brad/clust_92_outfiles/clust_92_stats.txt', 'epi2brad/clust_93_outfiles/clust_93_stats.txt', 'epi2brad/epi2brad_outfiles/epi2brad_stats.txt', 'epi2brad/clust_94_outfiles/clust_94_stats.txt', 'epi2brad/clust_88_outfiles/clust_88_stats.txt', 'epi2brad/clust_89_outfiles/clust_89_stats.txt', 'epi2brad/clust_82_outfiles/clust_82_stats.txt', 'epi2brad/clust_83_outfiles/clust_83_stats.txt']


In [75]:
# Another list containing the second set, the summary stats
sum_stats = []

infile=open(epifilenames[0], 'rt') # again, hard-coded for clustering threshold 91
counter=0
for line in infile:
    counter+=1
    if 'state  reads_raw' in line:
        stateline=counter
        print varline # not necessary but want to just check
        print line
    else:
        continue
     
infile.close() # because it's iterated through the whole file already so we need to close and reopen

infile=open(epifilenames[0], 'rt')

for lines in itertools.islice(infile, stateline, stateline+12):
    lines2 = lines.strip().split()
    lines2.append('91') # eventually change to whatever file it's on
    sum_stats.append(lines2)
        
print sum_stats

infile.close()

sumstats_labels=['sample', 'state', 'reads_raw', 'reads_passed', 'clust_total', 'clust_hidepth','hetero_est','error_est','reads_consens','loci_assembly','clust_threshold']
df_sumstats = pd.DataFrame.from_records(sum_stats, columns=sumstats_labels)

print df_sumstats

df_sumstats.to_csv("./epi2brad_clust_91_sumstats.csv")

77
                  state  reads_raw  reads_passed_filter  clusters_total  clusters_hidepth  hetero_est  error_est  reads_consens  loci_in_assembly

[['Ahah_R0089a_GTGA', '7', '899185', '899185', '263377', '187331', '0.053375', '0.027391', '186628', '57420', '91'], ['Ahah_R0089b_TCAG', '7', '647552', '647552', '208439', '127445', '0.040318', '0.027654', '126861', '53996', '91'], ['Ahah_R0090_GCTT', '7', '573267', '573267', '192735', '110106', '0.055877', '0.026345', '109569', '52834', '91'], ['Eant_T6857_ACCA', '7', '580913', '580913', '194899', '127422', '0.039920', '0.027235', '127034', '80158', '91'], ['Eant_T6859a_AGTG', '7', '668442', '668442', '239620', '153847', '0.038327', '0.026792', '153377', '84646', '91'], ['Eant_T6859b_CATC', '7', '776853', '776853', '250457', '184397', '0.044036', '0.026574', '183894', '92773', '91'], ['Ebou_R0153_CTAC', '7', '544642', '544642', '158842', '114067', '0.050679', '0.026216', '113697', '45577', '91'], ['Ebou_R0156_TGTC', '7', '554437', '5544